# L01: 競馬予想モデル学習

2種類のLightGBMモデルを構築・比較：
1. **オッズなしモデル**: 純粋な予測能力（レース前情報のみ）
2. **オッズありモデル**: 速報オッズを含む（穴馬発見用）

**実行環境**: ローカル（M4 MacBook Air）

---
## 1. 環境設定

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

PROJECT_ROOT = Path('../../')
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

RACES_PARQUET = PROJECT_ROOT / 'data' / 'processed' / 'races.parquet'
RESULTS_PARQUET = PROJECT_ROOT / 'data' / 'processed' / 'results.parquet'
MODEL_DIR = PROJECT_ROOT / 'models'
MODEL_DIR.mkdir(exist_ok=True)

print(f"データパス: {RACES_PARQUET}")

In [ ]:
races_df = pd.read_parquet(RACES_PARQUET)
results_df = pd.read_parquet(RESULTS_PARQUET)

print(f"レース数: {len(races_df):,}")
print(f"出走馬データ数: {len(results_df):,}")

---
## 2. データ分割

- **Train**: 2010〜2024年
- **Test**: 2025年〜

In [ ]:
results_df['race_date'] = pd.to_datetime(results_df['race_date'])
results_df['year'] = results_df['race_date'].dt.year

TRAIN_END_YEAR = 2024

train_df = results_df[results_df['year'] <= TRAIN_END_YEAR].copy()
test_df = results_df[results_df['year'] > TRAIN_END_YEAR].copy()

print(f"Train: {len(train_df):,} ({train_df['year'].min()}〜{train_df['year'].max()})")
print(f"Test: {len(test_df):,} ({test_df['year'].min()}〜{test_df['year'].max()})")

---
## 3. 特徴量エンジニアリング

In [ ]:
def calculate_historical_features(df, history_df):
    """各出走馬の過去成績から特徴量を計算"""
    history_df = history_df.sort_values('race_date')
    
    # 馬ごとの過去成績
    horse_stats = history_df.groupby('horse_id').agg({
        'finish_position': ['count', 'mean'],
        'race_id': 'count'
    }).reset_index()
    horse_stats.columns = ['horse_id', 'horse_race_count', 'horse_avg_finish', 'horse_total_races']
    
    win_stats = history_df.groupby('horse_id').apply(
        lambda x: pd.Series({
            'horse_win_rate': (x['finish_position'] == 1).sum() / len(x) if len(x) > 0 else 0,
            'horse_place_rate': (x['finish_position'] <= 3).sum() / len(x) if len(x) > 0 else 0,
        })
    ).reset_index()
    horse_stats = horse_stats.merge(win_stats, on='horse_id', how='left')
    
    # 騎手の成績
    jockey_stats = history_df.groupby('jockey_id').apply(
        lambda x: pd.Series({
            'jockey_race_count': len(x),
            'jockey_win_rate': (x['finish_position'] == 1).sum() / len(x) if len(x) > 0 else 0,
            'jockey_place_rate': (x['finish_position'] <= 3).sum() / len(x) if len(x) > 0 else 0,
        })
    ).reset_index()
    
    # 調教師の成績
    trainer_stats = history_df.groupby('trainer_id').apply(
        lambda x: pd.Series({
            'trainer_race_count': len(x),
            'trainer_win_rate': (x['finish_position'] == 1).sum() / len(x) if len(x) > 0 else 0,
            'trainer_place_rate': (x['finish_position'] <= 3).sum() / len(x) if len(x) > 0 else 0,
        })
    ).reset_index()
    
    result = df.merge(horse_stats, on='horse_id', how='left')
    result = result.merge(jockey_stats, on='jockey_id', how='left')
    result = result.merge(trainer_stats, on='trainer_id', how='left')
    
    return result

print("特徴量計算関数を定義")

In [ ]:
print("特徴量を計算中...")

all_history = results_df[results_df['year'] <= TRAIN_END_YEAR].copy()

train_features = calculate_historical_features(train_df, all_history)
test_features = calculate_historical_features(test_df, all_history)

print(f"Train features: {train_features.shape}")
print(f"Test features: {test_features.shape}")

In [ ]:
def add_features(df):
    df = df.copy()
    df['surface_encoded'] = df['surface'].map({'芝': 0, 'ダート': 1}).fillna(-1)
    df['target'] = (df['finish_position'] <= 3).astype(int)
    return df

train_features = add_features(train_features)
test_features = add_features(test_features)

print("特徴量追加完了")

---
## 4. モデル定義

### 特徴量の分類
- **レース前情報**: distance, surface, level_score, horse_number, gate_number, impost, 過去成績
- **レース直前情報**: popularity（人気）, odds（オッズ）

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

# オッズなしモデル（純粋予測）
FEATURE_COLS_NO_ODDS = [
    # レース情報（レース前に確定）
    'distance', 'surface_encoded', 'level_score',
    'horse_number', 'gate_number', 'impost',
    # 馬の過去成績
    'horse_race_count', 'horse_avg_finish',
    'horse_win_rate', 'horse_place_rate',
    # 騎手の成績
    'jockey_race_count', 'jockey_win_rate', 'jockey_place_rate',
    # 調教師の成績
    'trainer_race_count', 'trainer_win_rate', 'trainer_place_rate',
]

# オッズありモデル（速報オッズ込み）
FEATURE_COLS_WITH_ODDS = FEATURE_COLS_NO_ODDS + ['popularity', 'odds']

print("オッズなしモデル特徴量:", len(FEATURE_COLS_NO_ODDS))
print("オッズありモデル特徴量:", len(FEATURE_COLS_WITH_ODDS))

In [ ]:
# データ準備
train_clean = train_features.dropna(subset=['target'] + FEATURE_COLS_WITH_ODDS)
test_clean = test_features.dropna(subset=['target'] + FEATURE_COLS_WITH_ODDS)

print(f"Train (clean): {len(train_clean):,}")
print(f"Test (clean): {len(test_clean):,}")

---
## 5. モデル学習・評価

In [ ]:
def train_and_evaluate(train_data, test_data, feature_cols, model_name):
    """モデルの学習と評価"""
    print(f"\n{'='*50}")
    print(f"🎯 {model_name}")
    print(f"{'='*50}")
    
    X_train = train_data[feature_cols]
    y_train = train_data['target']
    X_test = test_data[feature_cols]
    y_test = test_data['target']
    
    # Validation split
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
    
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'random_state': 42,
    }
    
    train_ds = lgb.Dataset(X_tr, label=y_tr)
    val_ds = lgb.Dataset(X_val, label=y_val, reference=train_ds)
    
    model = lgb.train(
        params,
        train_ds,
        num_boost_round=500,
        valid_sets=[train_ds, val_ds],
        valid_names=['train', 'valid'],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=0)  # 静音
        ]
    )
    
    # 予測
    y_pred_proba = model.predict(X_test)
    
    # 評価
    auc = roc_auc_score(y_test, y_pred_proba)
    print(f"\n📊 Test AUC: {auc:.4f}")
    
    # レースごとの的中率
    test_with_pred = test_data.copy()
    test_with_pred['pred_proba'] = y_pred_proba
    
    def calc_hit_rate(group, top_n=3):
        top_preds = group.nlargest(top_n, 'pred_proba')
        return (top_preds['finish_position'] <= 3).any()
    
    race_hit = test_with_pred.groupby('race_id').apply(calc_hit_rate)
    hit_rate = race_hit.mean()
    print(f"🎯 上位3頭予測の的中率: {hit_rate:.1%}")
    
    # 回収率シミュレーション
    def simulate_returns(group):
        top = group.nlargest(1, 'pred_proba').iloc[0]
        return top['odds'] * 100 if top['finish_position'] == 1 else 0
    
    returns = test_with_pred.groupby('race_id').apply(simulate_returns)
    total_bet = len(returns) * 100
    total_return = returns.sum()
    return_rate = total_return / total_bet * 100
    print(f"💰 単勝回収率: {return_rate:.1f}%")
    
    # 特徴量重要度
    importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importance()
    }).sort_values('importance', ascending=False)
    
    return {
        'model': model,
        'auc': auc,
        'hit_rate': hit_rate,
        'return_rate': return_rate,
        'importance': importance,
        'predictions': test_with_pred
    }

In [ ]:
# オッズなしモデル
result_no_odds = train_and_evaluate(
    train_clean, test_clean,
    FEATURE_COLS_NO_ODDS,
    "オッズなしモデル（純粋予測）"
)

In [ ]:
# オッズありモデル
result_with_odds = train_and_evaluate(
    train_clean, test_clean,
    FEATURE_COLS_WITH_ODDS,
    "オッズありモデル（速報オッズ込み）"
)

---
## 6. モデル比較

In [ ]:
print("\n" + "="*60)
print("📊 モデル比較サマリー")
print("="*60)

comparison = pd.DataFrame({
    '指標': ['AUC', '的中率（上位3頭→3着内）', '単勝回収率'],
    'オッズなし': [
        f"{result_no_odds['auc']:.4f}",
        f"{result_no_odds['hit_rate']:.1%}",
        f"{result_no_odds['return_rate']:.1f}%"
    ],
    'オッズあり': [
        f"{result_with_odds['auc']:.4f}",
        f"{result_with_odds['hit_rate']:.1%}",
        f"{result_with_odds['return_rate']:.1f}%"
    ]
})

print(comparison.to_string(index=False))

In [ ]:
# 特徴量重要度の比較
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, (name, result) in zip(axes, [
    ('オッズなしモデル', result_no_odds),
    ('オッズありモデル', result_with_odds)
]):
    imp = result['importance'].head(10)
    ax.barh(imp['feature'], imp['importance'])
    ax.set_xlabel('Importance')
    ax.set_title(f'{name}\nTop 10 Features')
    ax.invert_yaxis()

plt.tight_layout()
plt.show()

---
## 7. モデル保存

In [ ]:
import pickle

# オッズなしモデル
with open(MODEL_DIR / 'model_no_odds.pkl', 'wb') as f:
    pickle.dump(result_no_odds['model'], f)

# オッズありモデル
with open(MODEL_DIR / 'model_with_odds.pkl', 'wb') as f:
    pickle.dump(result_with_odds['model'], f)

# 設定
config = {
    'feature_cols_no_odds': FEATURE_COLS_NO_ODDS,
    'feature_cols_with_odds': FEATURE_COLS_WITH_ODDS,
    'train_end_year': TRAIN_END_YEAR,
}

with open(MODEL_DIR / 'model_config.pkl', 'wb') as f:
    pickle.dump(config, f)

print("✅ モデル保存完了")
print(f"  - {MODEL_DIR / 'model_no_odds.pkl'}")
print(f"  - {MODEL_DIR / 'model_with_odds.pkl'}")
print(f"  - {MODEL_DIR / 'model_config.pkl'}")